In [1]:
import os
from pathlib import Path
import tifffile as tiff
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from tifffile import TiffFile, TiffWriter
from PIL import Image

### Pipeline
Chunking Function

In [18]:
def get_frame_count(tiff_path):
    """Helper function to count frames in an existing TIFF file."""
    try:
        with TiffFile(tiff_path) as tif:
            return len(tif.pages)
    except Exception as e:
        print(f"⚠️ Error reading {tiff_path}: {e}")
        return 0  # Return 0 to avoid errors if the file is unreadable

def split_tiff_stack(source_dir, output_dir, chunk_size):
    """
    Splits large TIFF stacks into smaller chunks of frames and performs a sanity check.

    Args:
        source_dir (str): Directory containing the source TIFF files.
        output_dir (str): Directory to save the chunked TIFF files.
        chunk_size (int): Number of frames per chunk.
    """
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    
    tiff_files = [f for f in os.listdir(source_dir) if f.endswith('.tif')]
    tiff_files.sort()  # Sort files alphabetically
    
    total_original_frames = 0
    total_chunked_frames = 0
    chunk_num = 1

    for tiff_file in tiff_files:
        source_path = os.path.join(source_dir, tiff_file)
        print(f"Processing: {tiff_file}")

        with TiffFile(source_path) as tif:
            total_frames = len(tif.pages)
            total_original_frames += total_frames
            print(f"Total frames in {tiff_file}: {total_frames}")

            for start in range(0, total_frames, chunk_size):
                end = min(start + chunk_size, total_frames)
                frames = range(start, end)
                chunk_frame_count = len(frames)  # Expected number of frames in the chunk

                chunk_filename = f"{os.path.splitext(tiff_file)[0]}_Chunk_{chunk_num:03d}.tif"
                chunk_path = os.path.join(output_dir, chunk_filename)

                # **Check if file already exists**
                if os.path.exists(chunk_path):
                    existing_frame_count = get_frame_count(chunk_path)
                    if existing_frame_count == chunk_frame_count:
                        print(f"🔄 Chunk {chunk_num} already exists and is valid. Skipping...")
                        total_chunked_frames += existing_frame_count  # Ensure count is correct
                        chunk_num += 1
                        continue
                    else:
                        print(f"⚠️ Existing chunk {chunk_num} has {existing_frame_count}/{chunk_frame_count} frames. Reprocessing...")

                try:
                    with TiffWriter(chunk_path, bigtiff=True) as writer:
                        for frame_index in frames:
                            try:
                                data = tif.pages[frame_index].asarray()
                                writer.write(data, photometric='minisblack', metadata=None)
                            except Exception as e:
                                print(f"⚠️ Error processing frame {frame_index}: {e}")

                    # **Sanity Check: Validate Chunked File**
                    with TiffFile(chunk_path) as chunk_tif:
                        chunked_frame_count = len(chunk_tif.pages)

                    if chunked_frame_count == chunk_frame_count:
                        print(f"✅ Chunk {chunk_num} verified: {chunked_frame_count}/{chunk_frame_count} frames")
                    else:
                        print(f"❌ Mismatch in Chunk {chunk_num}: {chunked_frame_count} frames instead of {chunk_frame_count}")

                    total_chunked_frames += chunked_frame_count
                    chunk_num += 1

                except Exception as e:
                    print(f"❌ Error creating chunk {chunk_num}: {e}")

    # **Final Sanity Check: Validate Total Frame Count**
    if total_chunked_frames == total_original_frames:
        print(f"\n✅ Sanity Check Passed: {total_chunked_frames}/{total_original_frames} frames processed successfully.")
    else:
        print(f"\n❌ Sanity Check Failed: Expected {total_original_frames}, but got {total_chunked_frames} frames.")


# Example Usage:
# source_directory = r"F:\videosDOI\f547_sample"
# output_directory = r"F:\videosDOI\Processed Videos\ChunkedTiffs_f547"
# frames_per_chunk = 900  # Number of frames per chunk

# split_tiff_stack(source_directory, output_directory, frames_per_chunk)


Cropping Function

In [4]:
def select_crop_region(image):
    """
    Allows the user to select a cropping region using OpenCV window.

    Args:
        image (numpy.ndarray): The first frame of the TIFF stack.

    Returns:
        tuple: (x, y, w, h) bounding box coordinates.
    """
    r = cv2.selectROI("Select ROI", image, fromCenter=False, showCrosshair=True)
    cv2.destroyAllWindows()
    return r  # (x, y, width, height)

def crop_tiff(input_tiff_path, output_tiff_path, crop_box, chunk_size=900):
    """
    Crops a TIFF file based on the provided crop_box and saves the output.

    Args:
        input_tiff_path (str): Path to input TIFF file.
        output_tiff_path (str): Path to save the cropped TIFF.
        crop_box (tuple): (x, y, w, h) coordinates for cropping.
        chunk_size (int): Number of frames to process before printing progress.
    """
    with TiffFile(input_tiff_path) as tif:
        total_frames = len(tif.pages)

        with TiffWriter(output_tiff_path, bigtiff=True) as writer:
            for frame_index in range(total_frames):
                # Read each frame
                data = tif.pages[frame_index].asarray()

                # Convert to PIL Image for cropping
                img = Image.fromarray(data)
                x, y, w, h = crop_box
                cropped_img = img.crop((x, y, x + w, y + h))

                # Convert back to NumPy array
                cropped_data = np.array(cropped_img)

                # Write the cropped frame to the new TIFF file
                writer.write(cropped_data, photometric='minisblack')

                # Print progress
                if (frame_index + 1) % chunk_size == 0 or frame_index + 1 == total_frames:
                    print(f"Processed {frame_index + 1}/{total_frames} frames.")

def auto_crop_tiffs_in_folder(input_folder, output_folder):
    """
    Crops all TIFF files in the input folder using a user-defined ROI from the first TIFF.
    
    Args:
        input_folder (str): Path to the folder containing TIFF files.
        output_folder (str): Path to the folder to save cropped TIFF files.
    """
    os.makedirs(output_folder, exist_ok=True)

    tiff_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.tif')])
    
    if not tiff_files:
        print(f"No TIFF files found in {input_folder}. Exiting...")
        return

    # Select region from the first TIFF file
    first_tiff_path = os.path.join(input_folder, tiff_files[0])
    with TiffFile(first_tiff_path) as tif:
        first_frame = tif.pages[0].asarray()

    print("Select cropping region on the first TIFF image...")
    crop_box = select_crop_region(first_frame)  # Get (x, y, w, h)

    # Apply the same crop to all TIFFs
    for tiff_file in tiff_files:
        input_tiff_path = os.path.join(input_folder, tiff_file)
        output_tiff_path = os.path.join(output_folder, tiff_file)

        print(f"Processing {tiff_file}...")
        crop_tiff(input_tiff_path, output_tiff_path, crop_box)

    print("Cropping completed for all TIFFs.")

# Example usage:
# input_folder = r"H:\videosDOI\Processed Videos\ChunkedTiffs_f547"
# output_folder = r"H:\videosDOI\Processed Videos\CroppedTiffs\f547"

# auto_crop_tiffs_in_folder(input_folder, output_folder)


Downsampling Function

In [5]:
import os
from pathlib import Path
import tifffile as tiff
import cv2
import numpy as np


def downsample_tiff(input_tiff_path, output_tiff_path, target_size=(512, 512)):
    # Sanity Check: Verify Input File Exists
    if not os.path.exists(input_tiff_path):
        print(f"❌ ERROR: Input TIFF file not found: {input_tiff_path}")
        return

    # Open the input TIFF file
    with TiffFile(input_tiff_path) as tif:
        num_frames = len(tif.pages)
        print(f"🔁 Processing {input_tiff_path}: {num_frames} frames")

    # Create a list to store downsampled frames
    downsampled_frames = []

    # Process each frame individually
    with TiffFile(input_tiff_path) as tif:
        for i, page in enumerate(tif.pages):
            frame = page.asarray()

            # Resize each frame to the target size
            resized_frame = cv2.resize(
                frame,
                target_size,
                interpolation=cv2.INTER_AREA
            )

            downsampled_frames.append(resized_frame)

            # Sanity Check: Ensure Correct Dimensions
            if resized_frame.shape != target_size[::-1]:  # (height, width)
                print(f"❌ ERROR: Downsampled frame {i+1} has incorrect size! Expected: {target_size}, Got: {resized_frame.shape}")

            # Progress update
            if (i + 1) % 100 == 0 or i == num_frames - 1:
                print(f"Processed {i + 1}/{num_frames} frames...")

    # Save the downsampled frames to a new TIFF file
    with TiffWriter(output_tiff_path, bigtiff=True) as tiff_writer:
        for frame in downsampled_frames:
            tiff_writer.write(frame)

    # Validate Output File
    with TiffFile(output_tiff_path) as output_tif:
        output_frames = len(output_tif.pages)
        output_shape = output_tif.pages[0].asarray().shape

    if output_frames == num_frames and output_shape == target_size[::-1]:
        print(f"✅ SUCCESS: Downsampled TIFF saved with {output_frames} frames ({target_size[0]}x{target_size[1]}) to {output_tiff_path}")
    else:
        print(f"❌ ERROR: Output TIFF validation failed! Frames: {output_frames}/{num_frames}, Size: {output_shape}/{target_size[::-1]}")


    
def batch_downsample_tiffs(input_folder, output_folder, target_size=(512, 512)):
    """
    Batch processes all TIFF files in a folder, downsampling them.

    Args:
        input_folder (str): Path to the folder containing input TIFF files.
        output_folder (str): Path to save the downsampled TIFF files.
        target_size (tuple): Target dimensions (width, height) for downsampling (default is 512x512).
    """
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)

    # Ensure the output folder exists
    output_folder.mkdir(parents=True, exist_ok=True)

    tiff_files = list(input_folder.glob("*.tif"))
    if not tiff_files:
        print(f"❌ ERROR: No TIFF files found in {input_folder}. Exiting...")
        return

    print(f"Found {len(tiff_files)} TIFF files in {input_folder}. Starting batch downsampling...")

    for tiff_file in tiff_files:
        input_tiff_path = tiff_file
        output_tiff_path = output_folder / f"{tiff_file.stem}_downsampled.tif"
        
        print(f"Processing file: {tiff_file.name}")
        downsample_tiff(input_tiff_path, output_tiff_path, target_size)

# Example Usage
# input_tiff_folder = r"F:\videosDOI\Processed Videos\CroppedTiffs\f547"
# output_tiff_folder = r"F:\videosDOI\Processed Videos\DownsampledTiffs\547"
# batch_downsample_tiffs(input_tiff_folder, output_tiff_folder, target_size=(512, 512))


Check if TIF files are readable

In [ ]:
def check_tiff_files(folder_path):
    """
    Checks all TIFF files in a folder to ensure they are not corrupt or unreadable.

    Args:
        folder_path (str): Path to the folder containing TIFF files.

    Returns:
        list: A list of corrupt or unreadable TIFF file paths.
    """
    # List all TIFF files in the folder
    tiff_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')]
    
    if not tiff_files:
        print(f"❌ No TIFF files found in the folder: {folder_path}")
        return []

    corrupt_files = []
    
    for tiff_file in tiff_files:
        try:
            # Attempt to open the TIFF file
            with TiffFile(tiff_file) as tif:
                # Ensure there are pages in the TIFF file
                if not tif.pages:
                    print(f"❌ ERROR: No pages found in {tiff_file}.")
                    corrupt_files.append(tiff_file)
                else:
                    # Read the first page to ensure the file is readable
                    tif.pages[0].asarray()
            print(f"✅ File is valid: {tiff_file}")
        except Exception as e:
            print(f"❌ ERROR: Corrupt or unreadable file: {tiff_file}. Error: {e}")
            corrupt_files.append(tiff_file)

    if corrupt_files:
        print("\n❌ Summary of Corrupt Files:")
        for file in corrupt_files:
            print(f" - {file}")
    else:
        print("\n✅ All TIFF files are valid.")

    return corrupt_files

# Example usage
# folder_path = r"F:\videosDOI\Processed Videos\DownsampledTiffs\547"
# corrupt_files = check_tiff_files(folder_path)


✅ File is valid: F:\videosDOI\Processed Videos\DownsampledTiffs\547\DOI_1_MMStack_Default.ome_Chunk_001_downsampled.tif
✅ File is valid: F:\videosDOI\Processed Videos\DownsampledTiffs\547\DOI_1_MMStack_Default.ome_Chunk_002_downsampled.tif
✅ File is valid: F:\videosDOI\Processed Videos\DownsampledTiffs\547\DOI_1_MMStack_Default.ome_Chunk_003_downsampled.tif
✅ File is valid: F:\videosDOI\Processed Videos\DownsampledTiffs\547\DOI_1_MMStack_Default_1.ome_Chunk_004_downsampled.tif
✅ File is valid: F:\videosDOI\Processed Videos\DownsampledTiffs\547\DOI_1_MMStack_Default_1.ome_Chunk_005_downsampled.tif
✅ File is valid: F:\videosDOI\Processed Videos\DownsampledTiffs\547\DOI_1_MMStack_Default_1.ome_Chunk_006_downsampled.tif
✅ File is valid: F:\videosDOI\Processed Videos\DownsampledTiffs\547\DOI_1_MMStack_Default_10.ome_Chunk_007_downsampled.tif
✅ File is valid: F:\videosDOI\Processed Videos\DownsampledTiffs\547\DOI_1_MMStack_Default_10.ome_Chunk_008_downsampled.tif
✅ File is valid: F:\videosDO

Convert to MP4

In [6]:
def tiff_to_mp4_bulk(tiff_dir, output_dir, fps=30):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # List all TIFF files in the directory
    tiff_files = [f for f in os.listdir(tiff_dir) if f.endswith('.tif')]
    tiff_files.sort()  # Sort files if necessary

    if not tiff_files:
        print(f"❌ ERROR: No TIFF files found in {tiff_dir}. Exiting...")
        return

    for tiff_file in tiff_files:
        start_time = time.time()

        # Paths for input and output
        tiff_path = os.path.join(tiff_dir, tiff_file)
        output_path = os.path.join(output_dir, f"{os.path.splitext(tiff_file)[0]}.mp4")

        print(f"📂 Processing: {tiff_file}")

        # Verify Input TIFF File Exists
        if not os.path.exists(tiff_path):
            print(f"❌ ERROR: TIFF file not found: {tiff_path}")
            continue

        # Read TIFF stack
        read_start = time.time()
        try:
            with TiffFile(tiff_path) as tif:
                if not tif.pages:
                    print(f"❌ ERROR: No frames found in {tiff_path}. Skipping...")
                    continue

                # Normalize frames for uint16 TIFFs to uint8
                frames = [
                    cv2.normalize(page.asarray(), None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                    for page in tif.pages
                ]
        except Exception as e:
            print(f"❌ ERROR: Failed to read TIFF file {tiff_file}: {e}")
            continue
        read_end = time.time()
        print(f"⏳ Time to read TIFF stack: {read_end - read_start:.2f} seconds")

        # Sanity Check: Validate Frames
        if len(frames) == 0:
            print(f"❌ ERROR: No valid frames found in {tiff_file}. Skipping...")
            continue

        # Get height and width from the first frame
        height, width = frames[0].shape
        print(f"✅ TIFF loaded successfully: {len(frames)} frames, {width}x{height} resolution")

        # Initialize video writer
        write_start = time.time()
        fourcc = cv2.VideoWriter_fourcc(*'H264')
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height), isColor=False)

        # Write frames to MP4
        frame_count = 0
        for frame in frames:
            try:
                # Convert grayscale to BGR for OpenCV compatibility
                frame_bgr = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
                video_writer.write(frame_bgr)
                frame_count += 1
            except Exception as e:
                print(f"❌ ERROR: Failed to process frame {frame_count} in {tiff_file}: {e}")

        # Release the video writer
        video_writer.release()
        write_end = time.time()
        print(f"⏳ Time to write video: {write_end - write_start:.2f} seconds")

        # Verify Output MP4
        if os.path.exists(output_path) and os.path.getsize(output_path) > 0:
            print(f"✅ SUCCESS: MP4 saved successfully ({output_path}, {frame_count} frames)")
        else:
            print(f"❌ ERROR: MP4 file was not created or is empty: {output_path}")

        # Total time for the file
        end_time = time.time()
        print(f"Total processing time for {tiff_file}: {end_time - start_time:.2f} seconds\n")


# Example usage
# tiff_directory = r"F:\videosDOI\Processed Videos\DownsampledTiffs\547"
# output_directory = r"F:\videosDOI\Processed Videos\ch_do_mp4_vids\547_sample"
# frames_per_second = 30

# tiff_to_mp4_bulk(tiff_directory, output_directory, frames_per_second)


Checking if the MP4 can be read

In [8]:
def check_video_integrity(video_path):
    """
    Checks if a video file is playable and reports any issues.

    Args:
        video_path (str): Path to the video file.

    Returns:
        dict: Contains the status and details of the video check.
    """
    result = {"file": video_path, "status": "OK", "details": ""}
    try:
        # Attempt to open the video file
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            result["status"] = "Corrupted"
            result["details"] = "Unable to open video file."
            return result

        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_count += 1

        cap.release()

        # Check if the video contains frames
        if frame_count == 0:
            result["status"] = "Corrupted"
            result["details"] = "No frames found in the video."
        else:
            result["details"] = f"Video is playable with {frame_count} frames."

    except Exception as e:
        result["status"] = "Corrupted"
        result["details"] = f"Exception occurred: {e}"

    return result

# Example usage
# video_directory = r"F:\videosDOI\Processed Videos\ch_do_mp4_vids\547_sample"
# video_files = [f for f in os.listdir(video_directory) if f.endswith('.mp4')]

# for video_file in video_files:
#     video_path = os.path.join(video_directory, video_file)
#     check_result = check_video_integrity(video_path)
#     print(f"File: {check_result['file']}, Status: {check_result['status']}, Details: {check_result['details']}")


Converting Cage m576

In [19]:
# chunk
source_directory = r"H:\videosDOI\m576"
output_directory_chunk = r"H:\videosDOI\Processed Videos\ChunkedTiffs\m576"
frames_per_chunk = 900  # Number of frames per chunk
split_tiff_stack(source_directory, output_directory_chunk, frames_per_chunk)

Processing: DOI_1_MMStack_Default.ome.tif
Total frames in DOI_1_MMStack_Default.ome.tif: 2041
🔄 Chunk 1 already exists and is valid. Skipping...
🔄 Chunk 2 already exists and is valid. Skipping...


<tifffile.TiffTag 270 @42040> coercing invalid ASCII to bytes, due to UnicodeDecodeError('charmap', b'1?<\xd0C\xfbJ\xe9N\xc2O\xa5QOSmS\xdbS\x1dU\x84R\xddR\x16SKVQT\xbeT\x90R\x82Q\xabQgQ\tO\x06QFQ\xe


🔄 Chunk 3 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_1.ome.tif
Total frames in DOI_1_MMStack_Default_1.ome.tif: 2041
🔄 Chunk 4 already exists and is valid. Skipping...
🔄 Chunk 5 already exists and is valid. Skipping...


<tifffile.TiffTag 270 @42040> coercing invalid ASCII to bytes, due to UnicodeDecodeError('charmap', b'.c,\x1b-0-\xe4+B,\xde.O/\xa01$3<243\x881d1m0\xfe2\xfb2\xf80\xbf0\x9b001d1\x870\xfa0\xb40\xfc2\x1


🔄 Chunk 6 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_10.ome.tif
Total frames in DOI_1_MMStack_Default_10.ome.tif: 2041
🔄 Chunk 7 already exists and is valid. Skipping...
🔄 Chunk 8 already exists and is valid. Skipping...


...

✅ Chunk 193 verified: 900/900 frames
🔄 Chunk 194 already exists and is valid. Skipping...
🔄 Chunk 195 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_68.ome.tif
Total frames in DOI_1_MMStack_Default_68.ome.tif: 2041
🔄 Chunk 196 already exists and is valid. Skipping...
🔄 Chunk 197 already exists and is valid. Skipping...
🔄 Chunk 198 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_69.ome.tif
Total frames in DOI_1_MMStack_Default_69.ome.tif: 2041
🔄 Chunk 199 already exists and is valid. Skipping...
🔄 Chunk 200 already exists and is valid. Skipping...
🔄 Chunk 201 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_7.ome.tif
Total frames in DOI_1_MMStack_Default_7.ome.tif: 2041
🔄 Chunk 202 already exists and is valid. Skipping...


<tifffile.TiffTag 270 @42040> coercing invalid ASCII to bytes, due to UnicodeDecodeError('charmap', b'\xd7\x12\r\x14\xa6\x13\x92\x13w\x13\xbd\x138\x13\xa7\x12*\x12\xaa\x12O\x12\xa7\x11O\x12\x90\x13\


🔄 Chunk 203 already exists and is valid. Skipping...
🔄 Chunk 204 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_70.ome.tif
Total frames in DOI_1_MMStack_Default_70.ome.tif: 2041
🔄 Chunk 205 already exists and is valid. Skipping...
🔄 Chunk 206 already exists and is valid. Skipping...
🔄 Chunk 207 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_71.ome.tif
Total frames in DOI_1_MMStack_Default_71.ome.tif: 2041
🔄 Chunk 208 already exists and is valid. Skipping...
🔄 Chunk 209 already exists and is valid. Skipping...
🔄 Chunk 210 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_72.ome.tif
Total frames in DOI_1_MMStack_Default_72.ome.tif: 1742


<tifffile.TiffTag 270 @42040> coercing invalid ASCII to bytes, due to UnicodeDecodeError('charmap', b'\xb31Q5\x11?_=\x03<\xff@\xf7<\xf7;\x14=7<Q=\xacCNHOFPC\xc8A\x90;.691M,\x89,q.\xcd4\xec:\xa4:r7\n


🔄 Chunk 211 already exists and is valid. Skipping...
🔄 Chunk 212 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_8.ome.tif
Total frames in DOI_1_MMStack_Default_8.ome.tif: 2041
🔄 Chunk 213 already exists and is valid. Skipping...
🔄 Chunk 214 already exists and is valid. Skipping...
🔄 Chunk 215 already exists and is valid. Skipping...
Processing: DOI_1_MMStack_Default_9.ome.tif
Total frames in DOI_1_MMStack_Default_9.ome.tif: 2041
🔄 Chunk 216 already exists and is valid. Skipping...
🔄 Chunk 217 already exists and is valid. Skipping...
🔄 Chunk 218 already exists and is valid. Skipping...

✅ Sanity Check Passed: 148694/148694 frames processed successfully.


In [20]:
# crop
input_folder = output_directory_chunk
output_folder_crop = r"H:\videosDOI\Processed Videos\CroppedTiffs\m576"
auto_crop_tiffs_in_folder(input_folder, output_folder_crop)

Select cropping region on the first TIFF image...
Processing DOI_1_MMStack_Default.ome_Chunk_001.tif...
Processed 900/900 frames.
Processing DOI_1_MMStack_Default.ome_Chunk_002.tif...
Processed 900/900 frames.
Processing DOI_1_MMStack_Default.ome_Chunk_003.tif...
Processed 241/241 frames.
Processing DOI_1_MMStack_Default_1.ome_Chunk_004.tif...
Processed 900/900 frames.
Processing DOI_1_MMStack_Default_1.ome_Chunk_005.tif...
Processed 900/900 frames.
Processing DOI_1_MMStack_Default_1.ome_Chunk_006.tif...
Processed 241/241 frames.
Processing DOI_1_MMStack_Default_10.ome_Chunk_007.tif...
Processed 900/900 frames.
Processing DOI_1_MMStack_Default_10.ome_Chunk_008.tif...
Processed 900/900 frames.
Processing DOI_1_MMStack_Default_10.ome_Chunk_009.tif...
Processed 241/241 frames.
Processing DOI_1_MMStack_Default_11.ome_Chunk_010.tif...
Processed 900/900 frames.
Processing DOI_1_MMStack_Default_11.ome_Chunk_011.tif...
Processed 900/900 frames.
Processing DOI_1_MMStack_Default_11.ome_Chunk_012

In [21]:
# downsample
input_tiff_folder = output_folder_crop
output_tiff_downsample = r"H:\videosDOI\Processed Videos\DownsampledTiffs\m576"
batch_downsample_tiffs(input_tiff_folder, output_tiff_downsample, target_size=(512, 512))

Found 218 TIFF files in H:\videosDOI\Processed Videos\CroppedTiffs\m576. Starting batch downsampling...
Processing file: DOI_1_MMStack_Default.ome_Chunk_001.tif
🔁 Processing H:\videosDOI\Processed Videos\CroppedTiffs\m576\DOI_1_MMStack_Default.ome_Chunk_001.tif: 900 frames
Processed 100/900 frames...
Processed 200/900 frames...
Processed 300/900 frames...
Processed 400/900 frames...
Processed 500/900 frames...
Processed 600/900 frames...
Processed 700/900 frames...
Processed 800/900 frames...
Processed 900/900 frames...
✅ SUCCESS: Downsampled TIFF saved with 900 frames (512x512) to H:\videosDOI\Processed Videos\DownsampledTiffs\m576\DOI_1_MMStack_Default.ome_Chunk_001_downsampled.tif
Processing file: DOI_1_MMStack_Default.ome_Chunk_002.tif
🔁 Processing H:\videosDOI\Processed Videos\CroppedTiffs\m576\DOI_1_MMStack_Default.ome_Chunk_002.tif: 900 frames
Processed 100/900 frames...
Processed 200/900 frames...
Processed 300/900 frames...
Processed 400/900 frames...
Processed 500/900 frames.

In [22]:
# convert
tiff_directory = output_tiff_downsample
output_directory = r"H:\videosDOI\Processed Videos\TifftoMP4\m576"
frames_per_second = 30
tiff_to_mp4_bulk(tiff_directory, output_directory, frames_per_second)

📂 Processing: DOI_1_MMStack_Default.ome_Chunk_001_downsampled.tif
⏳ Time to read TIFF stack: 2.70 seconds
✅ TIFF loaded successfully: 900 frames, 512x512 resolution
⏳ Time to write video: 1.55 seconds
✅ SUCCESS: MP4 saved successfully (H:\videosDOI\Processed Videos\TifftoMP4\m576\DOI_1_MMStack_Default.ome_Chunk_001_downsampled.mp4, 900 frames)
Total processing time for DOI_1_MMStack_Default.ome_Chunk_001_downsampled.tif: 4.26 seconds

📂 Processing: DOI_1_MMStack_Default.ome_Chunk_002_downsampled.tif
⏳ Time to read TIFF stack: 3.63 seconds
✅ TIFF loaded successfully: 900 frames, 512x512 resolution
⏳ Time to write video: 1.17 seconds
✅ SUCCESS: MP4 saved successfully (H:\videosDOI\Processed Videos\TifftoMP4\m576\DOI_1_MMStack_Default.ome_Chunk_002_downsampled.mp4, 900 frames)
Total processing time for DOI_1_MMStack_Default.ome_Chunk_002_downsampled.tif: 4.80 seconds

📂 Processing: DOI_1_MMStack_Default.ome_Chunk_003_downsampled.tif
⏳ Time to read TIFF stack: 0.93 seconds
✅ TIFF loaded su

Converting cage F589

In [24]:
# chunk
source_directory = r"H:\videosDOI\F589\12_20_24\DOI_1"
output_directory_chunk = r"H:\videosDOI\Processed Videos\ChunkedTiffs\F589"
frames_per_chunk = 900  # Number of frames per chunk
split_tiff_stack(source_directory, output_directory_chunk, frames_per_chunk)

Processing: DOI_2_MMStack_Default.ome.tif


<tifffile.TiffTag 270 @42035> coercing invalid ASCII to bytes, due to UnicodeDecodeError('charmap', b'\x13\x03\x01\x03\x10\x03\xe6\x02\xfa\x02\xd7\x02\xdc\x02\xec\x02\xf2\x02\xe9\x02\xad\x02\x9b\x02


Total frames in DOI_2_MMStack_Default.ome.tif: 2041
✅ Chunk 1 verified: 900/900 frames
✅ Chunk 2 verified: 900/900 frames
✅ Chunk 3 verified: 241/241 frames
Processing: DOI_2_MMStack_Default_1.ome.tif


<tifffile.TiffTag 270 @42035> coercing invalid ASCII to bytes, due to UnicodeDecodeError('charmap', b'\x06\x1c\x06H\x06S\x06#\x06\xc3\x05\xd6\x05O\x06\x0f\x06\xdd\x05\t\x06\xf4\x05\x12\x06\xb3\x05\x


Total frames in DOI_2_MMStack_Default_1.ome.tif: 2041
✅ Chunk 4 verified: 900/900 frames
✅ Chunk 5 verified: 900/900 frames
✅ Chunk 6 verified: 241/241 frames
Processing: DOI_2_MMStack_Default_10.ome.tif


...

Total frames in DOI_2_MMStack_Default_7.ome.tif: 2041
✅ Chunk 202 verified: 900/900 frames
✅ Chunk 203 verified: 900/900 frames
✅ Chunk 204 verified: 241/241 frames
Processing: DOI_2_MMStack_Default_70.ome.tif
Total frames in DOI_2_MMStack_Default_70.ome.tif: 2041
✅ Chunk 205 verified: 900/900 frames
✅ Chunk 206 verified: 900/900 frames
✅ Chunk 207 verified: 241/241 frames
Processing: DOI_2_MMStack_Default_71.ome.tif
Total frames in DOI_2_MMStack_Default_71.ome.tif: 2041
✅ Chunk 208 verified: 900/900 frames
✅ Chunk 209 verified: 900/900 frames
✅ Chunk 210 verified: 241/241 frames
Processing: DOI_2_MMStack_Default_72.ome.tif
Total frames in DOI_2_MMStack_Default_72.ome.tif: 72
✅ Chunk 211 verified: 72/72 frames
Processing: DOI_2_MMStack_Default_8.ome.tif


<tifffile.TiffTag 270 @42035> coercing invalid ASCII to bytes, due to UnicodeDecodeError('charmap', b'\xfb\n\xcc\n\xdd\n\xd7\n\x89\n\xc3\n\x84\n\xcd\n\xbb\n\x96\n\xef\n\xea\n\x90\n\x19\x0bz\x0b\x9a\


Total frames in DOI_2_MMStack_Default_8.ome.tif: 2041
✅ Chunk 212 verified: 900/900 frames
✅ Chunk 213 verified: 900/900 frames
✅ Chunk 214 verified: 241/241 frames
Processing: DOI_2_MMStack_Default_9.ome.tif


<tifffile.TiffTag 270 @42035> coercing invalid ASCII to bytes, due to UnicodeDecodeError('charmap', b'\xb9\x07M\x07a\x07m\x07T\x07\x98\x07\xf2\x07Q\x07\x16\x07z\x07\x91\x07\xdc\x07\x89\x07\x1a\x08\x


Total frames in DOI_2_MMStack_Default_9.ome.tif: 2041
✅ Chunk 215 verified: 900/900 frames
✅ Chunk 216 verified: 900/900 frames
✅ Chunk 217 verified: 241/241 frames

✅ Sanity Check Passed: 147024/147024 frames processed successfully.


In [25]:
# crop
input_folder = output_directory_chunk
output_folder_crop = r"H:\videosDOI\Processed Videos\CroppedTiffs\F589"
auto_crop_tiffs_in_folder(input_folder, output_folder_crop)

Select cropping region on the first TIFF image...
Processing DOI_2_MMStack_Default.ome_Chunk_001.tif...
Processed 900/900 frames.
Processing DOI_2_MMStack_Default.ome_Chunk_002.tif...
Processed 900/900 frames.
Processing DOI_2_MMStack_Default.ome_Chunk_003.tif...
Processed 241/241 frames.
Processing DOI_2_MMStack_Default_1.ome_Chunk_004.tif...
Processed 900/900 frames.
Processing DOI_2_MMStack_Default_1.ome_Chunk_005.tif...
Processed 900/900 frames.
Processing DOI_2_MMStack_Default_1.ome_Chunk_006.tif...
Processed 241/241 frames.
Processing DOI_2_MMStack_Default_10.ome_Chunk_007.tif...
Processed 900/900 frames.
Processing DOI_2_MMStack_Default_10.ome_Chunk_008.tif...
Processed 900/900 frames.
Processing DOI_2_MMStack_Default_10.ome_Chunk_009.tif...
Processed 241/241 frames.
Processing DOI_2_MMStack_Default_11.ome_Chunk_010.tif...
Processed 900/900 frames.
Processing DOI_2_MMStack_Default_11.ome_Chunk_011.tif...
Processed 900/900 frames.
Processing DOI_2_MMStack_Default_11.ome_Chunk_012

In [26]:
# downsample
input_tiff_folder = output_folder_crop
output_tiff_downsample = r"H:\videosDOI\Processed Videos\DownsampledTiffs\F589"
batch_downsample_tiffs(input_tiff_folder, output_tiff_downsample, target_size=(512, 512))

Found 217 TIFF files in H:\videosDOI\Processed Videos\CroppedTiffs\F589. Starting batch downsampling...
Processing file: DOI_2_MMStack_Default.ome_Chunk_001.tif
🔁 Processing H:\videosDOI\Processed Videos\CroppedTiffs\F589\DOI_2_MMStack_Default.ome_Chunk_001.tif: 900 frames
Processed 100/900 frames...
Processed 200/900 frames...
Processed 300/900 frames...
Processed 400/900 frames...
Processed 500/900 frames...
Processed 600/900 frames...
Processed 700/900 frames...
Processed 800/900 frames...
Processed 900/900 frames...
✅ SUCCESS: Downsampled TIFF saved with 900 frames (512x512) to H:\videosDOI\Processed Videos\DownsampledTiffs\F589\DOI_2_MMStack_Default.ome_Chunk_001_downsampled.tif
Processing file: DOI_2_MMStack_Default.ome_Chunk_002.tif
🔁 Processing H:\videosDOI\Processed Videos\CroppedTiffs\F589\DOI_2_MMStack_Default.ome_Chunk_002.tif: 900 frames
Processed 100/900 frames...
Processed 200/900 frames...
Processed 300/900 frames...
Processed 400/900 frames...
Processed 500/900 frames.

In [27]:
# convert
tiff_directory = output_tiff_downsample
output_directory = r"H:\videosDOI\Processed Videos\TifftoMP4\F589"
frames_per_second = 30
tiff_to_mp4_bulk(tiff_directory, output_directory, frames_per_second)

📂 Processing: DOI_2_MMStack_Default.ome_Chunk_001_downsampled.tif
⏳ Time to read TIFF stack: 3.27 seconds
✅ TIFF loaded successfully: 900 frames, 512x512 resolution
⏳ Time to write video: 1.29 seconds
✅ SUCCESS: MP4 saved successfully (H:\videosDOI\Processed Videos\TifftoMP4\F589\DOI_2_MMStack_Default.ome_Chunk_001_downsampled.mp4, 900 frames)
Total processing time for DOI_2_MMStack_Default.ome_Chunk_001_downsampled.tif: 4.56 seconds

📂 Processing: DOI_2_MMStack_Default.ome_Chunk_002_downsampled.tif
⏳ Time to read TIFF stack: 2.45 seconds
✅ TIFF loaded successfully: 900 frames, 512x512 resolution
⏳ Time to write video: 1.17 seconds
✅ SUCCESS: MP4 saved successfully (H:\videosDOI\Processed Videos\TifftoMP4\F589\DOI_2_MMStack_Default.ome_Chunk_002_downsampled.mp4, 900 frames)
Total processing time for DOI_2_MMStack_Default.ome_Chunk_002_downsampled.tif: 3.62 seconds

📂 Processing: DOI_2_MMStack_Default.ome_Chunk_003_downsampled.tif
⏳ Time to read TIFF stack: 0.83 seconds
✅ TIFF loaded su